In [7]:
import cv2
import joblib
import mediapipe as mp
import numpy as np

# Load the trained model
model_path = "../Dataset/Generated_Data/mlp_classifier_best_params.pkl"
mlp_model = joblib.load(model_path)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Load class labels
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
                'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'space']  # Adjust based on your dataset labels

def extract_hand_landmarks(image):
    """Extracts 21 hand landmarks from the given image."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0]
        data = []
        for landmark in landmarks.landmark:
            data.append(landmark.x)
            data.append(landmark.y)
        return np.array(data).reshape(1, -1)  # Reshape for model input
    return None

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip image for natural hand positioning
    frame = cv2.flip(frame, 1)

    # Extract hand landmarks
    landmarks = extract_hand_landmarks(frame)

    if landmarks is not None:
        # Make prediction
        prediction = mlp_model.predict(landmarks)
        predicted_label = prediction[0]

        # Draw label on the frame
        cv2.putText(frame, f'Prediction: {predicted_label}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the video feed
    cv2.imshow("ASL Hand Sign Prediction", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1740545736.998465 18090294 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1740545737.005649 18260012 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740545737.010845 18260023 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/anaconda3/envs/ASL-Translator/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/ASL-Translator/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/ASL-Translator/lib/python3.9/site-packages/sklearn/utils/validation.py:27

In [5]:
import cv2

def list_available_cameras(max_cameras=10):
    """Check available camera indices."""
    available_cameras = []

    for index in range(max_cameras):
        cap = cv2.VideoCapture(index)
        if cap.isOpened():
            available_cameras.append(index)
            cap.release()

    if available_cameras:
        print(f"Available cameras: {available_cameras}")
    else:
        print("No cameras found.")

# Run the function
list_available_cameras()


Available cameras: [0]


OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 2
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 3
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 4
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 5
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 6
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 7
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 8
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 9
OpenCV: camera failed to properly initialize!
